In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import random
import datetime as dt
import requests as rq
import pandas as pd

from job_client_graphql import Client
from jupyter_cell_display import CellDisplay

### Launch
+ **JOB**: Jupyter ObservableHQ bridge  
    see. [https://jupyter-observablehq-bridge/job-server-js-graphql](TBD)

In [3]:
s = Client(host='localhost',
           port=4001,
           path='/graphql',
           channel='channel-1',
           password='mysecret',
           debug=False)

In [4]:
url = 'https://raw.githubusercontent.com/ag-grid/ag-grid/master/grid-packages/ag-grid-docs/src/olympicWinnersSmall.json'
data = json.loads(rq.get(url).text)
print(f'nb lines = {len(data)}')


nb lines = 8618


In [5]:
df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8618 entries, 0 to 8617
Data columns (total 10 columns):
age        8613 non-null float64
athlete    8618 non-null object
bronze     8618 non-null int64
country    8618 non-null object
date       8618 non-null object
gold       8618 non-null int64
silver     8618 non-null int64
sport      8618 non-null object
total      8618 non-null int64
year       8618 non-null int64
dtypes: float64(1), int64(5), object(4)
memory usage: 673.4+ KB


## Write

In [34]:
def build_grid_options(df):
    col_defs = [{'field': c} for c in df.columns]
    grid_options = {
        'columnDefs': col_defs,
        'defaultColDef': {
            'editable': True,
            'sortable': True,
            'filter': True,
            'resizable': True
        },
        'rowSelection': 'single',
        'rowData': df.iloc[:100, :].to_dict(orient='records'),
    }
    return grid_options


In [35]:
grid_options = build_grid_options(df)
s.write('grid', json.dumps({'gridOptions': grid_options}))

True

## Open ObservableHQ notebook

In [36]:
url_nb = 'https://observablehq.com/d/9f2dfe3d673a3979'
url = url_nb+s.url_params
print(f'Open: {url}')

Open: https://observablehq.com/d/9f2dfe3d673a3979?host=localhost&port=4001&path=/graphql


## Subscribe

### DISPLAY

In [41]:
out = CellDisplay(name='subscribe-status', max_lines=8)
show_lines = []

def show(channel, name, value):
    ts = dt.datetime.now().strftime('%H:%M:%S.%f')[:-4]
    s = f'received on {channel}: {name} --> {json.loads(value)} ({ts})'
    out.update(s, kind='text', append=True)


received on channel-1: clicked --> {"age":21,"athlete":"Natalie Coughlin","bronze":1,"country":"United States","date":"29/08/2004","gold":2,"silver":2,"sport":"Swimming","total":5,"year":2004} (21:52:41.65)
received on channel-1: clicked --> {"age":26,"athlete":"Cindy Klassen","bronze":2,"country":"Canada","date":"26/02/2006","gold":1,"silver":2,"sport":"Speed Skating","total":5,"year":2006} (21:52:44.08)
received on channel-1: clicked --> {"age":21,"athlete":"Natalie Coughlin","bronze":1,"country":"United States","date":"29/08/2004","gold":2,"silver":2,"sport":"Swimming","total":5,"year":2004} (21:52:45.82)
received on channel-1: clicked --> {"age":22,"athlete":"Allison Schmitt","bronze":1,"country":"United States","date":"12/08/2012","gold":3,"silver":1,"sport":"Swimming","total":5,"year":2012} (21:52:49.14)
received on channel-1: clicked --> {'age': 22, 'athlete': 'Allison Schmitt', 'bronze': 1, 'country': 'United States', 'date': '12/08/2012', 'gold': 3, 'silver': 1, 'sport': 'Swi

In [42]:
out.display()


received on channel-1: clicked --> {"age":21,"athlete":"Natalie Coughlin","bronze":1,"country":"United States","date":"29/08/2004","gold":2,"silver":2,"sport":"Swimming","total":5,"year":2004} (21:52:41.65)
received on channel-1: clicked --> {"age":26,"athlete":"Cindy Klassen","bronze":2,"country":"Canada","date":"26/02/2006","gold":1,"silver":2,"sport":"Speed Skating","total":5,"year":2006} (21:52:44.08)
received on channel-1: clicked --> {"age":21,"athlete":"Natalie Coughlin","bronze":1,"country":"United States","date":"29/08/2004","gold":2,"silver":2,"sport":"Swimming","total":5,"year":2004} (21:52:45.82)
received on channel-1: clicked --> {"age":22,"athlete":"Allison Schmitt","bronze":1,"country":"United States","date":"12/08/2012","gold":3,"silver":1,"sport":"Swimming","total":5,"year":2012} (21:52:49.14)
received on channel-1: clicked --> {'age': 22, 'athlete': 'Allison Schmitt', 'bronze': 1, 'country': 'United States', 'date': '12/08/2012', 'gold': 3, 'silver': 1, 'sport': 'Swi

### START

In [43]:
ref = s.subscribe(action=show)

### STOP

In [44]:
s.subscribe_stop(ref)

subscribe ref 6nroIi stopped


## Read

In [45]:
name = 'clicked'
s.read(name)

'{"age":22,"athlete":"Allison Schmitt","bronze":1,"country":"United States","date":"12/08/2012","gold":3,"silver":1,"sport":"Swimming","total":5,"year":2012}'

## Read w/ Histo

In [46]:
name = 'clicked'
s.read_histo(name, 4, timestamp=True)

[{'value': '{"age":22,"athlete":"Allison Schmitt","bronze":1,"country":"United States","date":"12/08/2012","gold":3,"silver":1,"sport":"Swimming","total":5,"year":2012}',
  'timestamp': datetime.datetime(2020, 4, 14, 19, 52, 49, 140000)},
 {'value': '{"age":21,"athlete":"Natalie Coughlin","bronze":1,"country":"United States","date":"29/08/2004","gold":2,"silver":2,"sport":"Swimming","total":5,"year":2004}',
  'timestamp': datetime.datetime(2020, 4, 14, 19, 52, 45, 823000)},
 {'value': '{"age":26,"athlete":"Cindy Klassen","bronze":2,"country":"Canada","date":"26/02/2006","gold":1,"silver":2,"sport":"Speed Skating","total":5,"year":2006}',
  'timestamp': datetime.datetime(2020, 4, 14, 19, 52, 44, 79000)},
 {'value': '{"age":21,"athlete":"Natalie Coughlin","bronze":1,"country":"United States","date":"29/08/2004","gold":2,"silver":2,"sport":"Swimming","total":5,"year":2004}',
  'timestamp': datetime.datetime(2020, 4, 14, 19, 52, 41, 658000)}]

## Store

In [47]:
s.store

{'channel-1': {'clicked': '{"age":22,"athlete":"Allison Schmitt","bronze":1,"country":"United States","date":"12/08/2012","gold":3,"silver":1,"sport":"Swimming","total":5,"year":2012}'}}